In [1]:
import torch

from transformers import AutoTokenizer

#加载tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [2]:
from datasets import load_dataset

#加载数据集
dataset = load_dataset(path='lansinuote/ChnSentiCorp')

#编码
f = lambda x: tokenizer(
    x['text'], truncation=True, max_length=30, return_token_type_ids=False)
dataset = dataset.map(f, remove_columns=['text', 'label'])

#过滤句子长度
f = lambda x: len(x['input_ids']) >= 30
dataset = dataset.filter(f)


#重置label字段
def f(data):
    #定义第15个字为label
    data['label'] = data['input_ids'][15]

    #替换句子中的第15个字为mask
    data['input_ids'][15] = tokenizer.mask_token_id

    return data


dataset = dataset.map(f)

#设置数据类型
dataset.set_format('pt')

dataset, dataset['train'][0]

Map:   0%|          | 0/9286 [00:00<?, ? examples/s]

Map:   0%|          | 0/1158 [00:00<?, ? examples/s]

Map:   0%|          | 0/1157 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 9286
     })
     validation: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 1158
     })
     test: Dataset({
         features: ['input_ids', 'attention_mask', 'label'],
         num_rows: 1157
     })
 }),
 {'input_ids': tensor([ 101, 6848, 2885, 4403, 3736, 5709, 1736, 4638, 1333, 1728, 2218, 3221,
          3175,  912, 8024,  103, 4510, 1220, 2820, 3461, 4684, 2970, 1168, 6809,
          3862, 6804, 8024, 1453, 1741,  102]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]),
  'label': tensor(3300)})

In [3]:
loader = torch.utils.data.DataLoader(dataset['train'],
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

data = next(iter(loader))

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([8, 30])
attention_mask torch.Size([8, 30])
label torch.Size([8])


1160

In [4]:
#查看数据样例
for q, a in zip(data['input_ids'], data['label']):
    print(tokenizer.decode(q))
    print(tokenizer.decode(a))
    print('==============')

[CLS] 今 年 的 5. 1 、 10. 1 长 假 分 别 [MASK] 了 西 安 与 重 庆 的 如 家 ， 非 常 失 [SEP]
定
[CLS] 觉 得 很 多 书 都 比 这 书 好 ， 看 到 排 [MASK] 榜 的 排 名 才 买 它 的 ， 完 全 没 内 [SEP]
行
[CLS] 在 电 视 上 见 到 于 丹 ， 佩 服 她 的 口 [MASK] 和 思 维 ， 就 买 了 这 本 书 ， 但 有 [SEP]
才
[CLS] 内 存 小 ， 才 [UNK] ， 不 是 独 立 显 卡 ， [MASK] 摄 像 头 ， 硬 盘 不 是 [UNK] 。 接 口 布 [SEP]
无
[CLS] 真 正 说 起 来 ， 缺 点 还 是 不 少 的 。 [MASK] 先 ， 机 器 比 较 重 ， 价 格 不 便 宜 [SEP]
首
[CLS] 可 惜 机 器 wifi 不 支 持 wpa2 - psk, [MASK] 后 用 限 定 [UNK] 不 设 密 码 连 接 。 机 [SEP]
最
[CLS] 自 己 的 体 质 是 什 么 类 型 的 ， 该 怎 [MASK] 养 生 ， 看 了 这 本 书 让 我 一 下 子 [SEP]
样
[CLS] 性 价 比 不 用 说 了 ， 4455 买 的 ， 配 [MASK] 也 比 较 正 常 些 ， 工 作 娱 乐 看 电 [SEP]
置


In [5]:
#定义模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #加载预训练模型
        from transformers import AutoModel
        self.pretrained = AutoModel.from_pretrained(
            'google-bert/bert-base-chinese')

        self.fc = torch.nn.Linear(in_features=768,
                                  out_features=tokenizer.vocab_size)

    def forward(self, input_ids, attention_mask, label=None):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            last_hidden_state = self.pretrained(
                input_ids=input_ids,
                attention_mask=attention_mask).last_hidden_state

        #取第15个词的特征向量
        last_hidden_state = last_hidden_state[:, 15]

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(last_hidden_state).softmax(dim=1)

        #计算loss
        loss = None
        if label is not None:
            loss = torch.nn.functional.cross_entropy(out, label)

        return loss, out


model = Model()

model(**data)

(tensor(9.9583, grad_fn=<NllLossBackward0>),
 tensor([[6.2859e-05, 2.4186e-05, 3.1236e-05,  ..., 7.5827e-05, 5.1049e-05,
          5.9941e-05],
         [9.8141e-05, 2.9037e-05, 3.0801e-05,  ..., 6.9947e-05, 3.2658e-05,
          3.3411e-05],
         [6.8259e-05, 2.3807e-05, 3.0539e-05,  ..., 3.6393e-05, 4.8572e-05,
          4.5045e-05],
         ...,
         [7.5431e-05, 2.5728e-05, 3.0282e-05,  ..., 7.1031e-05, 8.0782e-05,
          3.5735e-05],
         [6.8091e-05, 1.9636e-05, 3.0080e-05,  ..., 1.0155e-04, 3.7200e-05,
          5.9929e-05],
         [5.9883e-05, 2.7097e-05, 4.7808e-05,  ..., 1.0384e-04, 8.2132e-05,
          3.7258e-05]], grad_fn=<SoftmaxBackward0>))

In [6]:
#执行训练
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(5):
        for i, data in enumerate(loader):
            loss, out = model(**data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % 200 == 0:
                out = out.argmax(dim=1)
                acc = (out == data['label']).sum().item() / len(data['label'])
                print(epoch, i, len(loader), loss.item(), acc)


train()

0 0 1160 9.958341598510742 0.0
0 200 1160 9.95827865600586 0.0
0 400 1160 9.956317901611328 0.0
0 600 1160 9.840551376342773 0.25
0 800 1160 9.957660675048828 0.0
0 1000 1160 9.902223587036133 0.125
1 0 1160 9.770970344543457 0.25
1 200 1160 9.732255935668945 0.375
1 400 1160 9.722811698913574 0.25
1 600 1160 9.78173542022705 0.25
1 800 1160 9.731805801391602 0.25
1 1000 1160 9.618192672729492 0.375
2 0 1160 9.832657814025879 0.125
2 200 1160 9.787606239318848 0.25
2 400 1160 9.686799049377441 0.375
2 600 1160 9.558889389038086 0.5
2 800 1160 9.681939125061035 0.25
2 1000 1160 9.501626968383789 0.5
3 0 1160 9.32802677154541 0.75
3 200 1160 9.61976146697998 0.5
3 400 1160 9.431528091430664 0.625
3 600 1160 9.483133316040039 0.625
3 800 1160 9.233800888061523 0.875
3 1000 1160 9.9302396774292 0.125
4 0 1160 9.478179931640625 0.5
4 200 1160 9.35655403137207 0.75
4 400 1160 9.726234436035156 0.375
4 600 1160 9.468263626098633 0.5
4 800 1160 9.402331352233887 0.625
4 1000 1160 9.34030151367

In [7]:
#执行测试
def test():
    loader_test = torch.utils.data.DataLoader(dataset['test'],
                                              batch_size=8,
                                              shuffle=True,
                                              drop_last=True)

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        with torch.no_grad():
            _, out = model(**data)

        out = out.argmax(dim=1)
        correct += (out == data['label']).sum().item()
        total += len(data['label'])

        print(i, len(loader_test), correct / total)

        if i == 5:
            break

    return correct / total


test()

0 144 0.125
1 144 0.375
2 144 0.375
3 144 0.40625
4 144 0.475
5 144 0.4791666666666667


0.4791666666666667